In [1]:
import pandas as pd

data = pd.read_csv('../Data/Advertising.csv')
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X = data[['TV', 'radio', 'newspaper']]
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

Mean Squared Error: 3.1740973539761064
R2 Score: 0.8994380241009119


In [4]:
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

Mean Squared Error: 2.798923009260352
R2 Score: 0.8966643473004003


In [6]:
model.fit(X, y)
coefficients = model.coef_
for feature, coef in zip(X.columns, coefficients):
    print(f'{feature}: {coef:.3f}')

TV: 0.046
radio: 0.189
newspaper: -0.001


In [13]:
import numpy as np

n = len(y)
p = X.shape[1]
residuals = y - y_pred
RSS = np.sum(residuals**2)
sigma2 = RSS / (n - p - 1)

X_with_intercept = np.column_stack((np.ones(n), X))
XtX_inv = np.linalg.inv(np.dot(X_with_intercept.T, X_with_intercept))
var_b = sigma2 * XtX_inv

std_errors = np.sqrt(np.diag(var_b))

for feature, std_err in zip(['intercept'] + list(X.columns), std_errors):
    print(f'{feature}: {std_err:.4f}')

intercept: 0.3127
TV: 0.0014
radio: 0.0086
newspaper: 0.0059


In [14]:
t_statistics = ([0] + coefficients) / std_errors
t_statistics

ValueError: operands could not be broadcast together with shapes (3,) (4,) 

In [15]:
coefficients.shape

(3,)

In [16]:
std_errors.shape

(4,)

In [17]:
([0] + coefficients).shape

(3,)

In [18]:
coefficients_std_errors = std_errors[1:]
t_statistics = coefficients / coefficients_std_errors
t_statistics

array([32.72178701, 21.83554865, -0.17624686])

In [20]:
from scipy import stats

p_values = 2 * (1 - stats.t.cdf(np.abs(t_statistics), n - p - 1))

In [21]:
p_values

array([0.        , 0.        , 0.86028198])

In [23]:
print ('Feature | Coefficient | Std. Error | t-statistic | p-value')
for feature, coef, se, t_stat, p_value in zip(list(X.columns), coefficients, std_errors[1:], t_statistics, p_values):
    print(f'{feature}: {coef:.3f} | {se:.4f} | {t_stat:.2f} | {p_value:.4f}')

Feature | Coefficient | Std. Error | t-statistic | p-value
TV: 0.046 | 0.0014 | 32.72 | 0.0000
radio: 0.189 | 0.0086 | 21.84 | 0.0000
newspaper: -0.001 | 0.0059 | -0.18 | 0.8603


### Question 1: Is there a relationship between sales and advertising budget?

Null hypothesis: $\beta_{TV} = \beta_{radio} = \beta_{newspaper} = 0$


Computed using the F-statistic:
$$F = \frac{(TSS - RSS) / p}{RSS / (n - p - 1)}$$

In [24]:
TSS = np.sum((y - y.mean())**2)
F = ((TSS - RSS) / p) / (RSS / (n - p - 1))
F

np.float64(566.9105401655801)

In [26]:
R2 = 1 - RSS / TSS
R2

np.float64(0.8966643473004003)

In [27]:
RSS

np.float64(559.7846018520704)

In [28]:
sigma2

np.float64(2.856043887000359)

In [29]:
from scipy import stats

p_value = stats.f.sf(F, p, n - p - 1)
p_value

np.float64(2.6473425434792483e-96)

p-value is negligible, almost zero. Therefor, we reject the null hypothesis with a very high degree of confidence.

### Question 2: How strong is the relationship?

In [30]:
RSE = np.sqrt(RSS / (n - p - 1))
RSE

np.float64(1.6899833984392743)

The standard error of the regression is 1.68. This means that the model's predictions are typically off of the population regression line by 1.68 units.

In [31]:
r2

0.8966643473004003

The multiple regression model explains 90% of the variance in sales.

### Question 3: Which media(s) contribute to sales?

To answer this, we do multiple hypothesis tests, each testing if the coefficient of a feature is zero.


In [48]:
model_all = LinearRegression()
model_all.fit(X, y)
y_pred_all = model_all.predict(X)
residuals_all = y - y_pred_all
RSS_all = np.sum(residuals_all**2)
TSS = np.sum((y - y.mean())**2)
n = len(y)
p = X.shape[1]

In [50]:
F_all = ((TSS - RSS_all) / p) / (RSS_all / (n - p - 1))
p_value_all = stats.f.sf(F_all, p, n - p - 1)
r2_all = 1 - RSS_all / TSS
print(f'RSS: {RSS_all}')
print(f'TSS: {TSS}')
print(f'F-statistic: {F_all}')
print(f'p-value: {p_value_all}')
print(f'R2: {r2_all}')

RSS: 556.8252629021872
TSS: 5417.14875
F-statistic: 570.2707036590942
p-value: 1.575227256092437e-96
R2: 0.8972106381789522


Null hypothesis 1: $\beta_{TV} = 0$


In [51]:
model_no_tv = LinearRegression()
model_no_tv.fit(X[['radio', 'newspaper']], y)
y_pred_no_tv = model_no_tv.predict(X[['radio', 'newspaper']])
residuals_no_tv = y - y_pred_no_tv
RSS_no_tv = np.sum(residuals_no_tv**2)
q = 2
F_tv = ((RSS_no_tv - RSS_all) / q) / (RSS_all / (n - p - 1))
p_value_tv = stats.f.sf(F_tv, q, n - p - 1)
r2_no_tv = 1 - RSS_no_tv / TSS
print(f'RSS_0: {RSS_no_tv}')
print(f'F-statistic: {F_tv}')
print(f'p-value: {p_value_tv}')
print(f'R2: {r2_no_tv}')

RSS_0: 3614.8352786449905
F-statistic: 538.2029184179415
p-value: 2.4421989083799917e-80
R2: 0.33270518395032256


p-value is negligible, we reject the null hypothesis, meaning that TV advertising has a significant effect on sales.

Null hypothesis 2: $\beta_{radio} = 0$

In [52]:
model_no_radio = LinearRegression()
model_no_radio.fit(X[['TV', 'newspaper']], y)
y_pred_no_radio = model_no_radio.predict(X[['TV', 'newspaper']])
residuals_no_radio = y - y_pred_no_radio
RSS_no_radio = np.sum(residuals_no_radio**2)
F_radio = ((RSS_no_radio - RSS_all) / q) / (RSS_all / (n - p - 1))
p_value_radio = stats.f.sf(F_radio, q, n - p - 1)
r2_no_radio = 1 - RSS_no_radio / TSS
print(f'RSS_0: {RSS_no_radio}')
print(f'F-statistic: {F_radio}')
print(f'p-value: {p_value_radio}')
print(f'R2: {r2_no_radio}')

RSS_0: 1918.5618118968275
F-statistic: 239.66258482226374
p-value: 2.2326838462402936e-53
R2: 0.6458354938293271


p-value is negligible, we reject the null hypothesis, meaning that radio advertising has a significant effect on sales.

Null hypothesis 3: $\beta_{newspaper} = 0$

In [53]:
model_no_newspaper = LinearRegression()
model_no_newspaper.fit(X[['TV', 'radio']], y)
y_pred_no_newspaper = model_no_newspaper.predict(X[['TV', 'radio']])
residuals_no_newspaper = y - y_pred_no_newspaper
RSS_no_newspaper = np.sum(residuals_no_newspaper**2)
F_newspaper = ((RSS_no_newspaper - RSS_all) / q) / (RSS_all / (n - p - 1))
p_value_newspaper = stats.f.sf(F_newspaper, q, n - p - 1)
r2_no_newspaper = 1 - RSS_no_newspaper / TSS
print(f'RSS_0: {RSS_no_newspaper}')
print(f'F-statistic: {F_newspaper}')
print(f'p-value: {p_value_newspaper}')
print(f'R2: {r2_no_newspaper}')

RSS_0: 556.9139800676184
F-statistic: 0.015614022551595014
p-value: 0.9845084687891822
R2: 0.8971942610828956


p-value is high, so we fail to reject the null hypothesis, meaning that newspaper advertising could have no significant effect on sales.